In [2]:
import sys
#sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [3]:
import os
import pandas as pd
os.chdir("..")
%pwd

'/Users/linus/projects/fdl/crisp'

In [4]:
from models.LinearInvariantRiskMinimization import LinearInvariantRiskMinimization
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [24]:
from synthetic.facebook_synthetic_data_generator import generator_example

dim_inv = 2
dim_spu = 2
n_exp_train = 6 #int(2e2)
n_exp_test = 6
n_env = 3
save_dir = 'data/synthetic'

In [6]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True,
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [7]:
LIRM_options = config.get("ensemble_options").get('LIRM', {})
LIRM_args = {
            "use_icp_initialization": False,
            "verbose": 1,
            "n_iterations": 3000, # 1000
            "seed": 0,
            "lr": 0.005, # 0.001
            "cuda": False
        }
LIRM_args.update(LIRM_options)

In [8]:
def get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir):
    # training data
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, False)
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env, False))

    # test data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, True)
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env, True))
    
    return outfile_train, outfile_test

In [31]:
df_train

,Causal_0,Causal_1,Non_causal_0,Non_causal_1,env_split,Subj_ID,Target
0,-0.006890,0.001798,-0.872451,-0.734993,0.0,0.0,2.0
1,-0.003430,-0.004423,-0.227449,-0.127548,0.0,1.0,2.0
2,-0.012709,-0.006520,-1.179085,-0.882011,1.0,2.0,2.0
3,0.006613,0.010388,0.717926,0.522778,1.0,3.0,1.0
4,0.012972,0.012450,0.591929,0.749831,2.0,4.0,1.0
5,0.000607,0.003996,-0.006603,-0.227203,2.0,5.0,0.0


In [18]:
results = {}

for n_example in [5]:#,2,3,4,5]:
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    
    outfile_train, outfile_test = get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir)
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    LIRM_args["target"] = config["data_options"]["targets"]
    LIRM_args["output_data_regime"] = output_data_regime
    LIRM_args["columns"] = test_dataset.predictor_columns

    lirm = LinearInvariantRiskMinimization(environment_datasets, val_dataset, test_dataset, LIRM_args)
    
    lirm_results_dict = lirm.results()
    results[n_example] = lirm_results_dict



n_example: 5 output_data_regime: multi-class 


Environments variables: {'E0': {'p': [0.8, 0.1, 0.1], 's': [0.3, 0.4, 0.3]}, 'E1': {'p': [0.9, 0.05, 0.05], 's': [0.4, 0.3, 0.3]}, 'E2': {'p': [0.97, 0.01, 0.02], 's': [0.3, 0.3, 0.4]}}
Generated Synthetic Data according to the Facebook setup Example: 5
Environments variables: {'E0': {'p': [0.8, 0.1, 0.1], 's': [0.3, 0.4, 0.3]}, 'E1': {'p': [0.9, 0.05, 0.05], 's': [0.4, 0.3, 0.3]}, 'E2': {'p': [0.97, 0.01, 0.02], 's': [0.3, 0.3, 0.4]}}
Generated Synthetic Data according to the Facebook setup Example: 5
Running a per sample experiment
Using synthetic dataset train/test split
Loaded  3  train environments
Env  0  has  1  samples
X shape  (1, 14)  y shape  (1, 1)
Env  1  has  1  samples
X shape  (1, 14)  y shape  (1, 1)
Env  2  has  1  samples
X shape  (1, 14)  y shape  (1, 1)
Loaded test set, X shape: (2, 14)  y shape:  (2, 1)
Start IRM training procedure


ValueError: Expected input batch_size (1) to match target batch_size (0).

In [49]:
print(results.get(n_example).keys())
"acc/r2:", results.get(n_example).get("test_acc")

dict_keys(['test_logits', 'test_acc', 'test_nll', 'test_probs', 'test_labels', 'feature_coeffients', 'to_bucket'])


('acc/r2:', tensor(0.2576))

In [51]:
result = results.get(n_example).get("to_bucket")

if output_data_regime == "multi-class":
    r = sorted(zip(result.get("features"), np.abs(result.get("coefficients")).sum(axis=1)), key=lambda x: x[1], reverse=True)
else:
    r = sorted(zip(result.get("features"), result.get("coefficients")), key=lambda x: abs(x[1]), reverse=True)
r

[('Causal_2', 28.2161705493927),
 ('Causal_1', 26.71197199821472),
 ('Causal_0', 26.41683316230774),
 ('Causal_3', 21.73872461915016),
 ('Non_causal_7', 2.7639010846614838),
 ('Non_causal_5', 2.7266255617141724),
 ('Non_causal_2', 2.3941598534584045),
 ('Non_causal_8', 2.0637218952178955),
 ('Non_causal_6', 1.5529405996203423),
 ('Non_causal_4', 1.2595409452915192),
 ('Non_causal_3', 1.1885599195957184),
 ('Non_causal_9', 0.9855049755424261),
 ('Non_causal_1', 0.6325856596231461),
 ('Non_causal_0', 0.30464520677924156)]